# <center><font face="arial" size="5" color=#0101DF>NUMERIQUE ET SCIENCES INFORMATIQUES 1ère NSI</font></center>

## <font color=#013ADF>Séquence N° 2 : projet STEGANOGRAPHIE</font>

**Problématique** :

Celle-ci est décrite sur le document de vision.

**Programme de décodage (lecture du message dans l'image)**

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: Gaspieux

Stéganographie : on cache une message se terminant par le caractère #
dans une image.
Le principe est le suivant : Comme l'oeil humain est peu sensible à une légère
nuance de couleur, on modifie éventuellement un bit de couleurs pour coder
le message.
Pour coder un 0, la composante de couleur qui le code doit avoir une valeur paire
Pour coder un 1, la composante de couleur qui la code doit avoir une valeur impaire
On vient ajuster la valeur si besoin.
Ici le programme de lecture du message dans l'image
"""

# importation de librairie
import sys
from PIL import Image


def ecriture_fichier(nom_fichier:str,message:str)->None:
    """
    Procédure d'écriture du message
    sur le disque, dans le répertoire courant
    """
    with open(nom_fichier, 'w', encoding="utf-8") as fichier_texte:
        fichier_texte.write(message)

def recup_couleurs_image(image_file)->list:
    """
    Cette fonction prend en entrée le nom de l'image 
    et renvoie une liste qui contient la valeur des 
    composantes rouge, vert et bleu de chaque pixel
    de l'image en décimal.
    """
    try:
        nom_image= Image.open(image_file)
    except:
        print ('Erreur sur ouverture du fichier ',image_file)
        sys.exit(1)
    largeur,hauteur=nom_image.size
    # balayage de l'image colonne par colonne de gauche à droite
    liste=[]
    for i in range(0,largeur):
        for j in range(0,hauteur):
            r,g,b = nom_image.getpixel((i,j))
            liste.append(r)
            liste.append(g)
            liste.append(b)
    nom_image.close()
    return liste

def decode_message(couleurs:list)->str:
    """
    lire les pixels de l'image si pair 0 sinon impair 1
    et renvoie le texte sans le caractère final #
    """
    text=""
    chaine=""
    cpt=0
    for i in couleurs:
        if i%2==0:
            chaine+='0'
        else:
            chaine+='1'
        cpt+=1
        if cpt==8:
            cpt=0
            nb_dec=0
            for power, digit in enumerate(chaine[::-1]):
                nb_dec+=int(digit)*(2**power)
            caractere=chr(nb_dec)
            if caractere=='#':
                return text
            text+=caractere
            chaine=""

#ouverture du fichier image
fichier_image = input('Indiquer le nom de l\'image dans laquelle le message est caché (nom + extension) : ')
fichier_texte=input('Indiquer le nom du fichier dans lequel écrire le message (nom + extension) : ')
# création d'une liste des couleurs des pixels de l'image
couleurs=recup_couleurs_image(fichier_image)
message_decode=decode_message(couleurs)
ecriture_fichier(fichier_texte,message_decode)
print("Traitement terminé !")

Indiquer le nom de l'image dans laquelle le message est caché (nom + extension) : Stallman.png
Indiquer le nom du fichier dans lequel écrire le message (nom + extension) : lecture.txt
Traitement terminé !


Un algorithme " gros grain" du programme ci-dessus serait de la forme :

- Ouvrir le fichier image qui contient le message ;

- Lire les composantes de couleurs des pixels de l'image ;

- En déduire la succession des bits (0 ou 1) ;

- Regrouper les bits par octet pour retrouver le caractère codé ;

- Arrêter le traitement si le caractère reconnu est "#" ;

- Écrire le message dans un fichier texte.

<div class="alert alert-warning">
<img src="Images/CR.png" alt="logo CR" width=5% align=right>

On vous demande d'étudier le programme pour comprendre comment le message est reconstitué. On détaillera plus particulièrement la fonction "decode_message" en traçant son algorithme. (**voir remarque en bas du document**)
</div>

**Voilà ce qui a pu être sauvé du programme de codage.**

In [1]:
from Codes.Alexandre import creation_image, recup_couleurs_image
from Codes.Malo import conv_message_vers_binaire, rgb
from Codes.Pierre_J import encode_message
from Codes.Pierre_M import lecture_fichier, infos_image
from Codes.Heremanu import creer_nom_sauvegarde
#Programme principal
fichier_image = input('Indiquer le nom de l\'image dans laquelle on cache le message (nom + extension) : ')
fichier_texte=input('Indiquer le nom du fichier dans lequel lire le message (nom + extension) : ')
# Modifie le mode du fichier de façon à ce qu'il soit en RGB
rgb(fichier_image)
# Récupère les caractéristiques de l'image originale
carac_image=infos_image(fichier_image)
# Création du nom de fichier de sauvegarde
fichier_sauvegarde=creer_nom_sauvegarde(fichier_image, carac_image)
# création d'une liste des couleurs des pixels de l'image
couleurs=recup_couleurs_image(fichier_image)
# Texte à cacher dans l'image. le caractère # indique que le message est terminé
# Ce message est dans un fichier texte appelé message.txt
texte=lecture_fichier(fichier_texte)
# création d'un chaine de bits à écrire sur l'image
chaine_binaire=conv_message_vers_binaire(texte)

if len(chaine_binaire)>len(couleurs):
    print("Le message est trop long pour l'image utilisée")
    
else:
    couleurs_mod=encode_message(chaine_binaire,couleurs)
    creation_image(carac_image, couleurs_mod, fichier_sauvegarde)
    print("Traitement terminé !")

Indiquer le nom de l'image dans laquelle on cache le message (nom + extension) : Stallman.png
Indiquer le nom du fichier dans lequel lire le message (nom + extension) : message.txt
Traitement terminé !


# carac_image

À la lecture du programme partiel de codage et d'après ce que nous connaissons de l'algorithme de décodage, nous pouvons déduire qu'il faut :

Écrire le message à cacher dans un fichier texte avec un éditeur de texte. Finir son écriture par le caractère # ;

Nous voyons également des appels à des fonctions qui d'après leurs noms, devraient réaliser les tâches précitées.

In [1]:
from PIL import Image
def infos_image(image_file)->tuple:
    """
    Renvoie les dimensions, le format (e.g. PNG)
    et le mode (e.g. RGB) de l'image
    originale si elle est trouvée !
    >>> infos_image("image.png")
    (772, 749, 'RGBA', 'PNG')
    """
    try:
        image = Image.open(image_file)
        imageproperties=((image.size), image.mode, image.format)
        return imageproperties
    except: 
        print("Erreur dans le traitement de l'image",image_file)

if __name__=="__main__":
    from doctest import testmod
    testmod()
    nomimage=input("Donner le nom de votre fichier image avec l'extension ")
    print(infos_image(nomimage))

**********************************************************************
File "__main__", line 7, in __main__.infos_image
Failed example:
    infos_image("image.png")
Expected:
    (772, 749, 'RGBA', 'PNG')
Got:
    Erreur dans le traitement de l'image image.png
**********************************************************************
1 items had failures:
   1 of   1 in __main__.infos_image
***Test Failed*** 1 failures.


KeyboardInterrupt: Interrupted by user

In [3]:
def recup_couleurs_image(image_file)->list:
    """
    Cette fonction prend en entrée le nom de l'image et
    renvoie une liste qui contient la valeur des composantes
    rouge, verte et bleue de chaque pixel de l'image en décimal.
    """
    try:
        nom_image= Image.open(image_file)
    except :
        print ('Erreur sur ouverture du fichier ',image_file)
    largeur,hauteur=nom_image.size
    # balayage de l'image colonne par colonne de gauche à droite
    liste=[]
    for i in range(0,largeur):
        for j in range(0,hauteur):
            r,g,b = nom_image.getpixel((i,j))
            liste.append(r)
            liste.append(g)
            liste.append(b)
    nom_image.close()
    return liste
recup_couleurs_image("Stallman.png")

Erreur sur ouverture du fichier  Stallman.png


UnboundLocalError: local variable 'nom_image' referenced before assignment

In [7]:
def lecture_fichier(nom_fichier)->str:
    """
    Fonction réalisant la lecture du fichier
    texte à écrire sur une image
    >>> lecture_fichier("texte.txt")
    'Ceci sera écrit à la fin de mon fichier !'
    """
    try:
        with open(nom_fichier, 'r', encoding="utf-8") as fichier_texte:
            lignes_texte=fichier_texte.read()
            return lignes_texte
    except: 
        print("Erreur dans le traitement du fichier", nom_fichier)
        
if __name__=="__main__":
    from doctest import testmod
    testmod()
    print(lecture_fichier(str(input("Donner le nom du fichier "))))

"àqÿàp\x12\x1f\x8c\\JBInHÜn6\x07\x1eFô\x88í\x1e'm±'i\x1c:%1ÅÚö\xa0Ýj \x9a-%Ñß\x08¢\tU%mªÔ\xad8ìx\x03\x8f\x89ã\x92OæÖv\x1fã\x89ÙÉ$H\x10\\\x92AÂU[m¶ñø©%mü\x9cqã\x8e\x07\x1cqÇn¯í$·\x89U91'\x1bj¶ÕR¶Õmª\xa0=\x92¥mrb^%\x01øçp\x01Ç~<¶¥"

In [ ]:
def conv_message_vers_binaire(texte:str)->str:
    """
    Cette fonction prend en entrée, le texte qui doit être caché dans l'image
    et renvoie une chaine de caractère qui contient le code binaire représentant
    chaque caractère du message à cacher sur 8 bits.
    >>> conv_message_vers_binaire("coucou les nullos")
    '0110001101101111011101010110001101101111011101010010000001101100011001010111001100100000011011100111010101101100011011000110111101110011'
    """
    assert type(texte) == str , ' L’entrée doit être un string '
    binMsg=''
    for carac in texte:
        temp=(bin(ord(carac))[2:])
        binMsg+=temp.zfill(8)
    return binMsg

if __name__=="__main__": 
    from doctest import testmod 
    testmod() 
    print(conv_message_vers_binaire("coucou les nullos"))

In [32]:
def encode_message(msg:str, coul:list)->list:
    """
    Cette fonction modifie les valeurs (couleurs) de
    la liste passée en paramètre selon le procédé suivant :
    si le nombre représentant la couleur est pair et le code binaire
    à écrire est 1, on doit rendre le nombre représentant la couleur impair.
    De même si le code binaire à écrire est 0 et que le nombre représentant
    la couleur est impair, on doit rendre le nombre représentant la couleur pair.
    >>> print(encode_message("0100",[1,254,122,123]))
    [0, 255, 122, 122]
    """
    assert len(msg)<=len(coul), "Le binaire du message doit être plus court que le nombre de couleurs."
    for bn in msg:
        assert bn=="0"or bn=="1", "Le message doit être en binaire"
    lst=[]
    
    for i in range(len(msg)):
        if msg[i]=="0" and coul[i]%2==1:
            lst.append(coul[i]-1)
        elif msg[i]=="1" and coul[i]%2==0:
            lst.append(coul[i]+1)
        else:
            lst.append(coul[i])
    return lst

if __name__=="__main__":
    from doctest import testmod
    testmod()
    #assert(encode_message("0100",[1,254,122,123]))==[0, 255, 122, 122]
    print(encode_message("0100",[1,254,122,123]))

[0, 255, 122, 122]


In [ ]:
def creation_image(caract:tuple, coul:list, image_file)->None:
    """
    Cette procédure crée une image ayant les mêmes
    caractéristiques que l'orignal mais avec les
    nouvelles couleurs, puis la sauvegarde avec
    un nom passé en argument
    """
    assert len(caract)==2, "la liste placé en arguments n'est pas une liste sortie de \"infos_images\""
    img_modifie=Image.new(caract[2], (caract[0][0], caract[0][1]))
    for x in range(image.height):
            for y in range(0, image.width*3, 3):          
                img_modifie.putpixel((x, y), (coul[y], coul[y+1], coul[y+2]))
    img_modifie.save(image_file)

In [ ]:
def creer_nom_sauvegarde(nom:str, caract:tuple)->str:
    """
    Fonction qui génère un nom pour la future image
    à sauvegarder, en relation avec le nom initial
    """
    assert type(nom)==str, "Il n'y a pas de nom pour le fichier"
    new_file = nom.replace("."+caract[1], "1."+caract[1])
    return new_file

<div class="alert alert-warning">
<img src="Images/CR.png" alt="logo CR" width=5% align=right>

Compléter les fonctions fournies de manière à reconstituer l'application et pouvoir coder et décoder les messages cachés dans les images.
</div>